In [2]:
import numpy as np
import pandas as pd
import random
import tensorflow as tf

from tensorflow import keras
from keras import backend as K
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from keras.layers import LSTM, Reshape, RepeatVector, TimeDistributed
from sklearn.model_selection import train_test_split
from keras.preprocessing.text import Tokenizer
from collections import Counter

import numpy as np
import os
from mido import MidiFile, MidiTrack, Message
import mido

Using TensorFlow backend.


In [27]:
paths = []
songs = []
for r, d, f in os.walk(r"C:\Users\Vee\Desktop\python\songs"):
    for file in f:
        if '.mid' in file:
            paths.append(os.path.join(r, file))

for path in paths:
    mid = MidiFile(path, type = 1)
    songs.append(mid)

In [28]:
notes = []
dataset = []

In [29]:
#for each in midi object in list of songs
for i in range(len(songs)):
    #for each note in midi object
    for msg in songs[i]:
        #filtering out meta messages
        if not msg.is_meta:
            #filtering out control changes
            if (msg.type == 'note_on') or (msg.type == 'note_off'):
                #normalizing note and velocity values
                notes.append([msg.note/127, msg.velocity/127, msg.time])
    #if more than 30 notes delete them
    if len(notes) > 60:
        for x in range(len(notes)-60):
            notes = np.delete(notes, 60, 0)
    #if less than 30 notes pad with zeros
    elif len(notes) < 60:
        for x in range(60 - len(notes)):
            notes.append([0,0,0])
    dataset.append(notes)    
    notes = []

In [30]:
dataset = np.array(dataset)
dataset.shape

(41, 60, 3)

In [31]:
bruv = []
#for each in midi object in list of songs
for i in range(len(songs)):
    #for each note in midi object
    for msg in songs[i]:
        #filtering out meta messages
        if not msg.is_meta:
            #filtering out control changes
            if (msg.type == 'note_on') or (msg.type == 'note_off'):
                #normalizing note and velocity values
                notes.append([msg.note/127, msg.velocity/127, msg.time])
    #if more than 40 notes delete them
    if len(notes) > 4:
        for x in range(len(notes)-4):
            notes = np.delete(notes, 4, 0)
    bruv.append(notes)    
    notes = []

In [32]:
bruv = np.array(bruv)
bruv.shape

(41, 4, 3)

In [33]:
#splitting data into train and test sets. 3/4 train, 1/4 test.
x_train,x_test,y_train,y_test = train_test_split(bruv, dataset, test_size=0.2, shuffle=False, random_state=42)

In [34]:
# define model
model = Sequential()
#shaping input to match data
model.add(LSTM(200, activation='relu', input_shape=(4, 3)))
#specifying output to have 60 timesteps
model.add(RepeatVector(60))
model.add(LSTM(200, activation='relu', return_sequences=True))
model.add(LSTM(200, activation='relu', return_sequences=True))
model.add(LSTM(200, activation='relu', return_sequences=True))
#specifying 3 features as the output
model.add(TimeDistributed(Dense(3)))
model.add(TimeDistributed(Dense(3)))
model.add(Dropout(0.2))
model.add(TimeDistributed(Dense(3)))
model.add(TimeDistributed(Dense(3)))
model.add(TimeDistributed(Dense(3)))
model.add(Activation('tanh'))
model.compile(loss='mean_absolute_error', optimizer='adam')
print(model.summary())

W0924 02:48:54.799951 19876 deprecation_wrapper.py:119] From D:\Anaconda\lib\site-packages\keras\backend\tensorflow_backend.py:66: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0924 02:48:54.811899 19876 deprecation_wrapper.py:119] From D:\Anaconda\lib\site-packages\keras\backend\tensorflow_backend.py:541: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0924 02:48:54.813912 19876 deprecation_wrapper.py:119] From D:\Anaconda\lib\site-packages\keras\backend\tensorflow_backend.py:4432: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0924 02:48:55.432758 19876 deprecation_wrapper.py:119] From D:\Anaconda\lib\site-packages\keras\backend\tensorflow_backend.py:148: The name tf.placeholder_with_default is deprecated. Please use tf.compat.v1.placeholder_with_default instead.

W0924 02:48:55.438743 19876 deprecation.py:506] From D:\Anaconda\lib\site-packages\keras\backend\t

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 200)               163200    
_________________________________________________________________
repeat_vector_1 (RepeatVecto (None, 60, 200)           0         
_________________________________________________________________
lstm_2 (LSTM)                (None, 60, 200)           320800    
_________________________________________________________________
lstm_3 (LSTM)                (None, 60, 200)           320800    
_________________________________________________________________
lstm_4 (LSTM)                (None, 60, 200)           320800    
_________________________________________________________________
time_distributed_1 (TimeDist (None, 60, 3)             603       
_________________________________________________________________
time_distributed_2 (TimeDist (None, 60, 3)            

In [35]:
model.fit(bruv, dataset, epochs=25, batch_size=10, verbose=2,validation_data=(x_test, y_test))

W0924 02:49:28.366307 19876 deprecation.py:323] From D:\Anaconda\lib\site-packages\tensorflow\python\ops\math_grad.py:1250: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
W0924 02:49:29.807084 19876 deprecation_wrapper.py:119] From D:\Anaconda\lib\site-packages\keras\backend\tensorflow_backend.py:1033: The name tf.assign_add is deprecated. Please use tf.compat.v1.assign_add instead.



Train on 41 samples, validate on 9 samples
Epoch 1/25
 - 3s - loss: 0.3381 - val_loss: 0.3077
Epoch 2/25
 - 0s - loss: 0.3282 - val_loss: 0.2902
Epoch 3/25
 - 0s - loss: 0.3020 - val_loss: 0.2322
Epoch 4/25
 - 0s - loss: 0.2622 - val_loss: 0.2106
Epoch 5/25
 - 0s - loss: 0.2405 - val_loss: 0.2027
Epoch 6/25
 - 0s - loss: 0.2314 - val_loss: 0.1877
Epoch 7/25
 - 0s - loss: 0.2225 - val_loss: 0.1885
Epoch 8/25
 - 0s - loss: 0.2175 - val_loss: 0.1882
Epoch 9/25
 - 0s - loss: 0.2114 - val_loss: 0.1841
Epoch 10/25
 - 0s - loss: 0.2094 - val_loss: 0.1775
Epoch 11/25
 - 0s - loss: 0.2072 - val_loss: 0.1800
Epoch 12/25
 - 0s - loss: 0.2033 - val_loss: 0.1773
Epoch 13/25
 - 0s - loss: 0.2007 - val_loss: 0.1732
Epoch 14/25
 - 0s - loss: 0.1969 - val_loss: 0.1721
Epoch 15/25
 - 0s - loss: 0.1965 - val_loss: 0.1720
Epoch 16/25
 - 0s - loss: 0.1964 - val_loss: 0.1692
Epoch 17/25
 - 0s - loss: 0.1916 - val_loss: 0.1696
Epoch 18/25
 - 0s - loss: 0.1932 - val_loss: 0.1682
Epoch 19/25
 - 0s - loss: 0.19

In [38]:
predict = model.predict(bruv)
#disregarding negative values
predict = abs(predict)

In [39]:
#adjusting from normalization
for y in range(len(songs)):
    for x in range(60):
        for i in range(2):
            predict[y][x][i] = predict[y][x][i] * 127
        predict[y][x][2] = predict[y][x][2] * 2870
        print(predict[y][x][2])

20.430546
11.322327
10.771173
18.188547
30.070358
41.696693
51.180145
57.867607
62.22925
64.19911
64.226715
63.122
61.481895
59.681267
57.944244
56.391308
55.077168
54.013504
53.186615
52.56837
52.12601
51.82541
51.634125
51.524662
51.47412
51.463696
51.478004
51.507557
51.543854
51.58229
51.618717
51.652157
51.680874
51.704834
51.7244
51.73988
51.751545
51.76047
51.76693
51.77179
51.7748
51.776802
51.778225
51.77878
51.77913
51.779366
51.778984
51.779118
51.7788
51.778942
51.77862
51.778687
51.778454
51.778286
51.77846
51.778095
51.7782
51.778305
51.77812
51.778156
14.575798
2.7997983
9.146912
2.8790922
13.024487
30.103504
44.32413
54.86152
61.544933
64.64702
64.62448
62.596745
59.772076
56.876965
54.291855
52.181988
50.584377
49.46196
48.74181
48.340202
48.176697
48.180897
48.29492
48.47198
48.67509
48.883377
49.081776
49.261406
49.418278
49.549488
49.65729
49.742405
49.808315
49.858505
49.895134
49.921604
49.939873
49.951645
49.959476
49.964157
49.966515
49.96782
49.96799
49.967876


In [40]:
from mido import Message, MidiFile, MidiTrack

mid = MidiFile()
track = MidiTrack()
mid.tracks.append(track)

track.append(Message('program_change', program=12, time=0))
x = 5
for i in range(60):
    track.append(Message('note_on', note= int(predict[x][i][0]), velocity=int(predict[x][i][1]), time=int(predict[x][i][2])))
    track.append(Message('note_off', note= int(predict[x][i][0]), velocity=int(predict[x][i][1]), time=int(predict[x][i][2])))
mid.save('xd.mid')

In [ ]:
from keras.models import load_model

# Creates a HDF5 file 'my_model.h5'
model.save('my_model.h5')